<a href="https://colab.research.google.com/github/alfinmaulani/skripsi2/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install happytransformer

In [3]:
import pandas as pd
import numpy as np
import happytransformer
import array

In [4]:
from happytransformer import HappyTextClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
korpus='/content/drive/MyDrive/fake news/beritapalsugabung.csv'
df=pd.read_csv(korpus)

In [1]:
df

NameError: ignored

In [7]:
happy_tc = HappyTextClassification(model_type="BERT", model_name="bert-base-uncased", num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
happy_tc.train(korpus)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 44898
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 134694
  Number of trainable parameters = 109483778
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.207400
1000,0.038000
1500,0.071100
2000,0.053600
2500,0.070200
3000,0.056400
3500,0.038300
4000,0.094500
4500,0.108100
5000,0.120100


Step,Training Loss
500,0.207400
1000,0.038000
1500,0.071100
2000,0.053600
2500,0.070200
3000,0.056400
3500,0.038300
4000,0.094500
4500,0.108100
5000,0.120100
